In [1]:
import mat_properties as prop


import os
import time
import cotel
import GTU
import mat_properties as prop
import numpy as n
import pandas as pd
from scipy.optimize import root


# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
fractiongas_KU = (
    0.710320591016015,
    0.00996710270335893,
    0.090538556815177,
    0.180531273012258,
    0.00864247645319178
)

gas_KU = prop.Materials_prop(
    gasmix,
    fractiongas_KU,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


Methane = prop.Materials_prop(
    "METHANE",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


fractionwaterMethane = (0.833372660622383, 0.166627339377617, 0, 0, 0)
waterMethanemix = "Water*METHANE"

waterMethane = prop.Materials_prop(
    waterMethanemix,
    fractionwaterMethane,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)

gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)


from PKM import steam_transformer


#пар из КУ в реформер как из аспена (убрать)
water_streams.loc['DROSVD-ST','T':'G'] = [509.0071373,7.845,3423.525641,47.55878321]

#паротрансформатор
ST = steam_transformer(stream11 = 'DROSVD-ST', 
                       water = water, 
                       water_streams = water_streams, 
                       heaters = heaters, 
                       Pdr1 = 2, 
                       Pdr2 = 0.8, 
                       P2 = 2, 
                       dT = 15, 
                       dTmin = 5, 
                       Tdec = 10)
steam_trans = ST.calc()

#Ввод в табл выходов из паротрансформатора
water_streams.loc['ST-GPK','T':'G'] = [steam_trans['T16'],steam_trans['P16'],steam_trans['H16'],steam_trans['G1']]
water_streams.loc['ST-PKM','T':'G'] = [steam_trans['T24'],steam_trans['P2'],steam_trans['H24'],steam_trans['G2']]
heaters.at['Strans','Qw'] = steam_trans['Q']

#реформер
from PKM import reformer
ref = reformer(stream11 = 'ST-PKM',
               water = water,
               gas_KU = gas_KU,
               Methane = Methane,
               waterMethane = waterMethane,
               water_streams = water_streams,
               heaters = heaters,
               Tref = 700,
               Pref = 2,
               T1gas = 1968.58395330148,
               T2gas = 800)
reform = ref.calc()

#Ввод в табл выходов из паротрансформатора

syngas_streams.loc['REF-COOL','T':'G'] = [reform['Tref'],reform['Pref'],reform['Hsg'],reform['Gref']]
heaters.at['Reformer','Qw'] = reform['Q']
syngas_streams.loc['REF-COOL','N2':'CO'] = list(reform['SGfrac'].values())

#Копирование с ГТУ, потом уберу
gas_streams.loc['GTU-SHESH','T':'G']   = gas_streams.loc['GTU-KU','T':'G']
gas_streams.loc['GTU-SHESH','N2':'Ar'] = gas_streams.loc['GTU-KU','N2':'Ar']

#Газы реформера
gas_streams.loc['AIR-REF','T':'G']     = [15,0.1,414.38,reform['Gair']]
gas_streams.loc['CH4-REF','T':'G']     = [15,0.7,881.50,reform['Gch4']]
gas_streams.loc['REF-SMESH','T':'G']   = [800,0.1,reform['H2gas'],reform['Ggas']]
gas_streams.loc['REF-SMESH','N2':'Ar'] = list(reform['Gasfrac'].values())

#Смешение
gas_streams.loc['SMESH-KU','G']   = gas_streams.at['REF-SMESH','G'] + gas_streams.at['GTU-SHESH','G']
gas_streams.loc['SMESH-KU','H']   = (gas_streams.at['REF-SMESH','G']*gas_streams.at['REF-SMESH','H']+gas_streams.at['GTU-SHESH','G']*gas_streams.at['GTU-SHESH','H'])/gas_streams.loc['SMESH-KU','G']
gas_streams.loc['SMESH-KU','P']   = 0.1



# gas_streams.loc['SMESH-KU','N2']  = (gas_streams.at['REF-SMESH','G']*gas_streams.at['REF-SMESH','N2']+gas_streams.at['GTU-SHESH','G']*gas_streams.at['GTU-SHESH','N2']*(0.0140067))/gas_streams.loc['SMESH-KU','G']
# gas_streams.loc['SMESH-KU','O2']  = (gas_streams.at['REF-SMESH','G']*gas_streams.at['REF-SMESH','O2']+gas_streams.at['GTU-SHESH','G']*gas_streams.at['GTU-SHESH','O2'])/gas_streams.loc['SMESH-KU','G']
# gas_streams.loc['SMESH-KU','CO2'] = (gas_streams.at['REF-SMESH','G']*gas_streams.at['REF-SMESH','CO2']+gas_streams.at['GTU-SHESH','G']*gas_streams.at['GTU-SHESH','CO2'])/gas_streams.loc['SMESH-KU','G']
# gas_streams.loc['SMESH-KU','H2O'] = (gas_streams.at['REF-SMESH','G']*gas_streams.at['REF-SMESH','H2O']+gas_streams.at['GTU-SHESH','G']*gas_streams.at['GTU-SHESH','H2O'])/gas_streams.loc['SMESH-KU','G']
# gas_streams.loc['SMESH-KU','Ar']  = (gas_streams.at['REF-SMESH','G']*gas_streams.at['REF-SMESH','Ar']+gas_streams.at['GTU-SHESH','G']*gas_streams.at['GTU-SHESH','Ar'])/gas_streams.loc['SMESH-KU','G']


Sr_mol_mass1 =  (gas_streams.at['REF-SMESH','N2']*(0.0280134) +
                gas_streams.at['REF-SMESH','O2']*(0.03199806) +
                gas_streams.at['REF-SMESH','CO2']*(0.0440095) +
                gas_streams.at['REF-SMESH','H2O']*(0.01801528) +
                gas_streams.at['REF-SMESH','Ar']*(0.039948))

Sr_mol_mass2 =  (gas_streams.at['GTU-SHESH','N2']*(0.0280134) +
                gas_streams.at['GTU-SHESH','O2']*(0.03199806) +
                gas_streams.at['GTU-SHESH','CO2']*(0.0440095) +
                gas_streams.at['GTU-SHESH','H2O']*(0.01801528) +
                gas_streams.at['GTU-SHESH','Ar']*(0.039948))
mol_flow_1 = gas_streams.at['REF-SMESH','G']/Sr_mol_mass1
mol_flow_2 = gas_streams.at['GTU-SHESH','G']/Sr_mol_mass2

mol_flow1_N2 = mol_flow_1*gas_streams.at['REF-SMESH','N2']
mol_flow1_O2 = mol_flow_1*gas_streams.at['REF-SMESH','O2']
mol_flow1_CO2 = mol_flow_1*gas_streams.at['REF-SMESH','CO2']
mol_flow1_H2O = mol_flow_1*gas_streams.at['REF-SMESH','H2O']
mol_flow1_Ar = mol_flow_1*gas_streams.at['REF-SMESH','Ar']

mol_flow2_N2 = mol_flow_2*gas_streams.at['GTU-SHESH','N2']
mol_flow2_O2 = mol_flow_2*gas_streams.at['GTU-SHESH','O2']
mol_flow2_CO2 = mol_flow_2*gas_streams.at['GTU-SHESH','CO2']
mol_flow2_H2O = mol_flow_2*gas_streams.at['GTU-SHESH','H2O']
mol_flow2_Ar = mol_flow_2*gas_streams.at['GTU-SHESH','Ar']

mol_flowsumm_N2 = mol_flow1_N2 + mol_flow2_N2
mol_flowsumm_O2 = mol_flow1_O2 + mol_flow2_O2
mol_flowsumm_CO2 = mol_flow1_CO2 + mol_flow2_CO2
mol_flowsumm_H2O = mol_flow1_H2O + mol_flow2_H2O
mol_flowsumm_Ar = mol_flow1_Ar + mol_flow2_Ar
mol_flow_summ = mol_flow_1 + mol_flow_2


gas_streams.loc['SMESH-KU','N2']  = mol_flowsumm_N2/mol_flow_summ
gas_streams.loc['SMESH-KU','O2']  = mol_flowsumm_O2/mol_flow_summ
gas_streams.loc['SMESH-KU','CO2'] = mol_flowsumm_CO2/mol_flow_summ
gas_streams.loc['SMESH-KU','H2O'] = mol_flowsumm_H2O/mol_flow_summ
gas_streams.loc['SMESH-KU','Ar']  = mol_flowsumm_Ar/mol_flow_summ




gas_SMESH = prop.Materials_prop(
    gasmix,
   (gas_streams.loc['SMESH-KU','N2'],
    gas_streams.loc['SMESH-KU','O2'],
    gas_streams.loc['SMESH-KU','CO2'],
    gas_streams.loc['SMESH-KU','H2O'],
    gas_streams.loc['SMESH-KU','Ar']),
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)


gas_streams.loc['SMESH-KU','T'] = gas_SMESH.p_h(gas_streams.loc['SMESH-KU','P'], gas_streams.loc['SMESH-KU','H'])['T']



gas_streams



,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.100000,0.1,958.869199,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
GTU-PEVD,542.100000,0.1,958.869199,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
PEVD-IVD,468.050000,0.1,500.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
IVD-EVD,309.200000,0.1,400.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
EVD-PPND,235.900000,0.1,300.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
PPND-IND,233.000000,0.1,200.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
IND-GPK,175.500000,0.1,100.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
GPK-out,106.900000,0.1,20.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
GTU-SHESH,542.100000,0.1,958.869199,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500


In [2]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,4976.537205,46.091916,0,1.960109e-22,0.063567,0,0.536615,0.050042,0.325899,0.023878


In [3]:
heaters

,Qw,Qg,KPD
PEVD,NaN,NaN,NaN
IVD,NaN,NaN,NaN
EVD,NaN,NaN,NaN
PPND,NaN,NaN,NaN
IND,NaN,NaN,NaN
GPK,NaN,NaN,NaN
SP2,NaN,NaN,NaN
SP1,NaN,NaN,NaN
OD,NaN,NaN,NaN
ASW,NaN,NaN,NaN


In [4]:
Sr_mol_mass1 =  (gas_streams.at['REF-SMESH','N2']*(0.0280134) +
                gas_streams.at['REF-SMESH','O2']*(0.03199806) +
                gas_streams.at['REF-SMESH','CO2']*(0.0440095) +
                gas_streams.at['REF-SMESH','H2O']*(0.01801528) +
                gas_streams.at['REF-SMESH','Ar']*(0.039948))

Sr_mol_mass2 =  (gas_streams.at['GTU-SHESH','N2']*(0.0280134) +
                gas_streams.at['GTU-SHESH','O2']*(0.03199806) +
                gas_streams.at['GTU-SHESH','CO2']*(0.0440095) +
                gas_streams.at['GTU-SHESH','H2O']*(0.01801528) +
                gas_streams.at['GTU-SHESH','Ar']*(0.039948))
mol_flow_1 = gas_streams.at['REF-SMESH','G']/Sr_mol_mass1
mol_flow_2 = gas_streams.at['GTU-SHESH','G']/Sr_mol_mass2

mol_flow1_N2 = mol_flow_1*gas_streams.at['REF-SMESH','N2']
mol_flow1_O2 = mol_flow_1*gas_streams.at['REF-SMESH','O2']
mol_flow1_CO2 = mol_flow_1*gas_streams.at['REF-SMESH','CO2']
mol_flow1_H2O = mol_flow_1*gas_streams.at['REF-SMESH','H2O']
mol_flow1_Ar = mol_flow_1*gas_streams.at['REF-SMESH','Ar']

mol_flow2_N2 = mol_flow_2*gas_streams.at['GTU-SHESH','N2']
mol_flow2_O2 = mol_flow_2*gas_streams.at['GTU-SHESH','O2']
mol_flow2_CO2 = mol_flow_2*gas_streams.at['GTU-SHESH','CO2']
mol_flow2_H2O = mol_flow_2*gas_streams.at['GTU-SHESH','H2O']
mol_flow2_Ar = mol_flow_2*gas_streams.at['GTU-SHESH','Ar']

mol_flowsumm_N2 = mol_flow1_N2 + mol_flow2_N2
mol_flowsumm_O2 = mol_flow1_O2 + mol_flow2_O2
mol_flowsumm_CO2 = mol_flow1_CO2 + mol_flow2_CO2
mol_flowsumm_H2O = mol_flow1_H2O + mol_flow2_H2O
mol_flowsumm_Ar = mol_flow1_Ar + mol_flow2_Ar
mol_flow_summ = mol_flow_1 + mol_flow_2


gas_streams.loc['SMESH-KU','N2']  = mol_flowsumm_N2/mol_flow_summ
gas_streams.loc['SMESH-KU','O2']  = mol_flowsumm_O2/mol_flow_summ
gas_streams.loc['SMESH-KU','CO2'] = mol_flowsumm_CO2/mol_flow_summ
gas_streams.loc['SMESH-KU','H2O'] = mol_flowsumm_H2O/mol_flow_summ
gas_streams.loc['SMESH-KU','Ar']  = mol_flowsumm_Ar/mol_flow_summ



gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,542.100000,0.1,958.869199,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
GTU-PEVD,542.100000,0.1,958.869199,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
PEVD-IVD,468.050000,0.1,500.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
IVD-EVD,309.200000,0.1,400.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
EVD-PPND,235.900000,0.1,300.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
PPND-IND,233.000000,0.1,200.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
IND-GPK,175.500000,0.1,100.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
GPK-out,106.900000,0.1,20.000000,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
GTU-SHESH,542.100000,0.1,958.869199,503.800000,0.780300,0.123700,0.030100,0.059400,0.006500
